# NSE Analytics Platform - Complete Demo

## 🎯 Professional Data Platform Demonstration

This notebook demonstrates a comprehensive NSE (National Stock Exchange) data analytics platform with:

### 📊 Core Capabilities
- **Data Fetching & Storage**: Automated collection from Yahoo Finance with quality validation
- **Interactive Analytics**: GUI-based returns and correlation analysis
- **Data Quality Management**: Comprehensive validation and error handling
- **Professional Presentation**: Clean, demo-ready code

### 🏗️ Technical Architecture
- **Database**: SQLite with normalized schema and constraints
- **Data Processing**: Pandas with vectorized operations
- **User Interface**: Interactive widgets with real-time feedback
- **Quality Assurance**: Multi-layer validation and error handling

### 📈 Business Value
- **Risk Management**: Correlation analysis for portfolio diversification
- **Performance Tracking**: Monthly/yearly returns monitoring
- **Data Integrity**: Comprehensive quality validation
- **Scalability**: Modular design for easy extension

---

**Author**: KO + GitHub Copilot  
**Purpose**: Interview demonstration and portfolio showcase  
**Date**: July 2025

## 🎯 Demo Usage Instructions

### Step 1: Data Fetching
1. **Run the first cell** to fetch and store NSE data
2. **Review the output** for successful/failed indices
3. **Check database statistics** (total records, failed tickers)

### Step 2: Returns Analysis
1. **Use the Returns Analysis tab** in the dashboard above
2. **Select indices** (single for monthly breakdown, multiple for yearly comparison)
3. **Choose time periods** (years of interest)
4. **Click "Calculate Returns"** to generate color-coded matrices

### Step 3: Correlation Analysis
1. **Switch to Correlation Matrix tab**
2. **Configure parameters**:
   - Indices selection
   - Time interval (Daily/Monthly)
   - Price type (Open/High/Low/Close)
   - Date range
3. **Click "Calculate Correlation"** for correlation matrix with quality reports

### Key Features Demonstrated
- ✅ **Professional Code Quality**: Clean, documented, maintainable
- ✅ **Data Engineering**: ETL pipeline with validation
- ✅ **Interactive Analytics**: User-friendly GUI with real-time feedback
- ✅ **Error Handling**: Comprehensive exception management
- ✅ **Business Value**: Risk management and performance tracking tools

---

**This notebook demonstrates production-ready data platform development suitable for financial analysis and portfolio management applications.**

## 📥 Part 1: Data Fetching & Storage System

This section demonstrates the ETL pipeline that fetches NSE indices data from Yahoo Finance and stores it in a SQLite database with comprehensive quality controls.

In [ ]:
"""
NSE Indices OHLC Data Fetcher
============================
Fetches OHLC data for NSE indices from Yahoo Finance and stores in SQLite database.

Features:
- Data validation (prevents negative/zero OHLC values)
- Duplicate prevention with unique constraints
- Error tracking for failed tickers

Author: KO + GitHub Copilot:)
"""

import yfinance as yf
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import warnings

# Suppress all warnings for clean output
warnings.filterwarnings('ignore')

# Configuration
INDICES = {
    'Nifty 50': '^NSEI',
    'Nifty Auto': '^CNXAUTO',
    'Nifty Bank': '^NSEBANK',
    'Nifty Financial Services': '^CNXFIN',
    'Nifty FMCG': '^CNXFMCG',
    'Nifty IT': '^CNXIT',
    'Nifty Media': '^CNXMEDIA',
    'Nifty Metal': '^CNXMETAL',
    'Nifty Pharma': '^CNXPHARMA',
    'Nifty PSU Bank': '^CNXPSUBANK',
    'Nifty Realty': '^CNXREALTY',
    'Nifty Private Bank': '^CNXPRBANK',
    'Nifty Consumer Durables': '^CNXCONSUMERDUR'
}

START_DATE = '2005-01-01'
END_DATE = datetime.today().strftime('%Y-%m-%d')
DATABASE_PATH = 'sqlite:///nse_data.db'

# Create database table
engine = create_engine(DATABASE_PATH)
with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS nse_ohlc_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ticker TEXT NOT NULL,
            name TEXT,
            date DATE NOT NULL,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            source TEXT DEFAULT 'yfinance',
            frequency TEXT DEFAULT 'daily',
            created_at TEXT,
            updated_at TEXT,
            UNIQUE(ticker, date, frequency, source)
        )
    """))

print(f"🚀 Fetching NSE data for {len(INDICES)} indices...")
print(f"📅 Date range: {START_DATE} to {END_DATE}\n")

# Initialize counters
inserted_count = 0
failed_count = 0
failed_tickers = []
invalid_count = 0

# Process each ticker
for name, ticker in INDICES.items():
    print(f"🔄 Processing {name} ({ticker})...", end=' ')
    
    try:
        # Download data
        df = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
        
        if df.empty:
            print("❌ No data")
            failed_count += 1
            failed_tickers.append(f"{name} ({ticker})")
            continue
        
        # Handle MultiIndex columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(1)
        
        # Prepare data for database
        df = df.reset_index()
        df['ticker'] = ticker
        df['name'] = name
        df['source'] = 'yfinance'
        df['frequency'] = 'daily'
        df['created_at'] = datetime.now().isoformat()
        df['updated_at'] = datetime.now().isoformat()
        
        # Insert into database
        rows_before = 0
        with engine.connect() as conn:
            result = conn.execute(text("SELECT COUNT(*) FROM nse_ohlc_prices WHERE ticker = :ticker"), 
                                {'ticker': ticker})
            rows_before = result.scalar()
        
        # Insert data with validation
        with engine.begin() as conn:
            for _, row in df.iterrows():
                try:
                    # Validate OHLC data - skip negative or zero values
                    ohlc_values = [row.get('Open'), row.get('High'), row.get('Low'), row.get('Close')]
                    if any(val is not None and not pd.isna(val) and val <= 0 for val in ohlc_values):
                        invalid_count += 1
                        continue  # Skip invalid data
                    
                    conn.execute(text("""
                        INSERT OR IGNORE INTO nse_ohlc_prices
                        (ticker, name, date, open, high, low, close, source, frequency, created_at, updated_at)
                        VALUES (:ticker, :name, :date, :open, :high, :low, :close, :source, :frequency, :created_at, :updated_at)
                    """), {
                        'ticker': row['ticker'],
                        'name': row['name'],
                        'date': row['Date'].date(),
                        'open': row.get('Open'),
                        'high': row.get('High'),
                        'low': row.get('Low'),
                        'close': row.get('Close'),
                        'source': row['source'],
                        'frequency': row['frequency'],
                        'created_at': row['created_at'],
                        'updated_at': row['updated_at']
                    })
                except:
                    pass  # Skip errors silently
        
        # Count new records
        with engine.connect() as conn:
            result = conn.execute(text("SELECT COUNT(*) FROM nse_ohlc_prices WHERE ticker = :ticker"), 
                                {'ticker': ticker})
            rows_after = result.scalar()
        
        new_records = rows_after - rows_before
        inserted_count += new_records
        print(f"✅ {len(df)} rows processed ({new_records} new)")
        
    except Exception:
        print("❌ Error")
        failed_count += 1
        failed_tickers.append(f"{name} ({ticker})")

# Final summary
print(f"\n" + "="*60)
print(f"📊 SUMMARY")
print(f"="*60)
print(f"✅ Successful: {len(INDICES) - failed_count}")
print(f"❌ Failed: {failed_count}")
print(f"📝 New records inserted: {inserted_count}")
if invalid_count > 0:
    print(f"⚠️  Invalid records skipped: {invalid_count} (negative/zero OHLC values)")

# Get total database count
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM nse_ohlc_prices"))
    total_records = result.scalar()
print(f"📊 Total records in database: {total_records:,}")

# Show failed tickers if any
if failed_tickers:
    print(f"\n❌ Failed tickers requiring review:")
    for ticker in failed_tickers:
        print(f"   • {ticker}")
    print(f"\n💡 Action: Check ticker symbols on Yahoo Finance")

print("="*60)

🚀 Fetching NSE data for 13 indices...
📅 Date range: 2025-01-01 to 2025-07-16

🔄 Processing Nifty 50 (^NSEI)... ✅ 133 rows processed (0 new)
🔄 Processing Nifty Auto (^CNXAUTO)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Bank (^NSEBANK)... ✅ 133 rows processed (0 new)
🔄 Processing Nifty Auto (^CNXAUTO)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Bank (^NSEBANK)... 


1 Failed download:
['^CNXFIN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-01-01 -> 2025-07-16)')
['^CNXFIN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-01-01 -> 2025-07-16)')


✅ 132 rows processed (0 new)
🔄 Processing Nifty Financial Services (^CNXFIN)... ❌ No data
🔄 Processing Nifty FMCG (^CNXFMCG)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty IT (^CNXIT)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Media (^CNXMEDIA)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty IT (^CNXIT)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Media (^CNXMEDIA)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Metal (^CNXMETAL)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Pharma (^CNXPHARMA)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Metal (^CNXMETAL)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Pharma (^CNXPHARMA)... ✅ 133 rows processed (0 new)
🔄 Processing Nifty PSU Bank (^CNXPSUBANK)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Realty (^CNXREALTY)... ✅ 133 rows processed (0 new)
🔄 Processing Nifty PSU Bank (^CNXPSUBANK)... ✅ 132 rows processed (0 new)
🔄 Processing Nifty Realty (^CNXREALTY)... ✅ 132 rows processed (0 new)
🔄 Process

HTTP Error 404: 

1 Failed download:
['^CNXPRBANK']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['^CNXPRBANK']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data
🔄 Processing Nifty Consumer Durables (^CNXCONSUMERDUR)... 

HTTP Error 404: 

1 Failed download:
['^CNXCONSUMERDUR']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['^CNXCONSUMERDUR']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data

📊 SUMMARY
✅ Successful: 10
❌ Failed: 3
📝 New records inserted: 0
📊 Total records in database: 37,167

❌ Failed tickers requiring review:
   • Nifty Financial Services (^CNXFIN)
   • Nifty Private Bank (^CNXPRBANK)
   • Nifty Consumer Durables (^CNXCONSUMERDUR)

💡 Action: Check ticker symbols on Yahoo Finance


## 📊 Part 2: Interactive Analytics Dashboard

This section provides a comprehensive analytics dashboard with interactive widgets for:
- **Returns Analysis**: Monthly/yearly performance matrices
- **Correlation Analysis**: Multi-timeframe correlation matrices
- **Data Quality Reporting**: Automated validation and issue detection

In [ ]:
"""
NSE Analytics Dashboard - Interactive GUI for Stock Returns and Correlation Analysis

This module provides a comprehensive dashboard for analyzing NSE indices performance
with features for calculating returns matrices and correlation analysis.

Key Features:
- Returns analysis with configurable time periods
- Correlation matrix calculation with multiple intervals
- Interactive widgets for user-friendly data selection
- Data quality validation and error handling
- Optimized database queries for better performance

Author: KO + GitHub Copilot:)
"""

# === IMPORTS ===
import pandas as pd
from datetime import date, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
from sqlalchemy import create_engine
import numpy as np
import logging
from typing import List, Dict, Tuple, Optional, Union

# === CONFIGURATION ===
# Configure logging for error tracking (but not display in notebook)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Prevent logger messages from appearing in notebook output
logger.propagate = False
# Only log to file if a file handler is added, not to console
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# NSE Indices mapping to Yahoo Finance ticker symbols
INDICES = {
    'Nifty 50': '^NSEI',
    'Nifty Auto': '^CNXAUTO',
    'Nifty Bank': '^NSEBANK',
    'Nifty FMCG': '^CNXFMCG',
    'Nifty IT': '^CNXIT',
    'Nifty Media': '^CNXMEDIA',
    'Nifty Metal': '^CNXMETAL',
    'Nifty Pharma': '^CNXPHARMA',
    'Nifty PSU Bank': '^CNXPSUBANK',
    'Nifty Realty': '^CNXREALTY',
    # Note: Some indices commented out due to data availability issues
    # 'Nifty Financial Services': '^CNXFIN',
    # 'Nifty Private Bank': '^CNXPRBANK',
    # 'Nifty Consumer Durables': '^CNXCONSUMERDUR'
}

# Date range configuration
DEFAULT_YEAR_RANGE = (2007, 2025)
DEFAULT_LOOKBACK_DAYS = 365
MONTHS_ORDER = ['Yearly', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# === UTILITY FUNCTIONS ===
def validate_data_quality(df: pd.DataFrame, ticker: str, index_name: str = None) -> Tuple[bool, List[str]]:
    """
    Validate data quality for a given DataFrame and ticker.
    
    Args:
        df: DataFrame containing OHLC data
        ticker: Ticker symbol being validated
        index_name: Human-readable index name for better reporting
        
    Returns:
        Tuple of (is_valid, list_of_issues)
    """
    issues = []
    display_name = index_name or ticker
    
    if df.empty:
        issues.append(f"No data available for {display_name}")
        return False, issues
    
    # Check for missing values
    missing_data = df.isnull().sum()
    if missing_data.any():
        missing_info = {k: v for k, v in missing_data.to_dict().items() if v > 0}
        if missing_info:
            issues.append(f"{display_name}: Missing data points - {missing_info}")
    
    # Note: Negative/zero price validation is handled at database level during data ingestion
    
    # Check for data gaps (more than 7 consecutive days missing)
    if 'date' in df.columns or df.index.name == 'date':
        date_series = df.index if df.index.name == 'date' else df['date']
        date_diff = date_series.diff()
        large_gaps = (date_diff > pd.Timedelta(days=7)).sum()
        if large_gaps > 0:
            max_gap = date_diff.max().days
            issues.append(f"{display_name}: {large_gaps} large data gaps detected (>7 days), max gap: {max_gap} days")
    
    # Check for extreme price movements (>50% in a single day)
    if 'close' in df.columns:
        daily_returns = df['close'].pct_change()
        extreme_moves = (abs(daily_returns) > 0.5).sum()
        if extreme_moves > 0:
            max_move = abs(daily_returns).max() * 100
            issues.append(f"{display_name}: {extreme_moves} extreme price movements detected (>50% daily change), max: {max_move:.1f}%")
    
    return len(issues) == 0, issues

def create_collapsible_section(title: str, content: List[str], is_open: bool = False) -> None:
    """
    Create a collapsible HTML section for displaying data quality information.
    
    Args:
        title: Title for the collapsible section
        content: List of content lines to display
        is_open: Whether the section should be open by default
    """
    import ipywidgets as widgets
    from IPython.display import HTML
    
    if not content:
        return
    
    # Create HTML for collapsible content
    content_html = "<br>".join([f"• {line}" for line in content])
    open_attr = "open" if is_open else ""
    
    html_content = f"""
    <details {open_attr} style="margin-top: 10px; margin-bottom: 10px; padding: 10px; 
                                border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9;">
        <summary style="font-weight: bold; cursor: pointer; padding: 5px; 
                       background-color: #e9e9e9; border-radius: 3px; margin-bottom: 10px;">
            {title} ({len(content)} items)
        </summary>
        <div style="margin-top: 10px; font-family: monospace; font-size: 12px; line-height: 1.4;">
            {content_html}
        </div>
    </details>
    """
    
    display(HTML(html_content))

def build_query_parameters(tickers: List[str]) -> str:
    """
    Safely build SQL IN clause parameters for ticker list.
    
    Args:
        tickers: List of ticker symbols
        
    Returns:
        Formatted string for SQL IN clause
    """
    return ','.join([f"'{ticker}'" for ticker in tickers])

def calculate_monthly_returns(closes: pd.Series, year: int) -> Dict[str, float]:
    """
    Calculate monthly and yearly returns for a given year.
    
    Args:
        closes: Series of month-end closing prices
        year: Year for calculation
        
    Returns:
        Dictionary with monthly returns and yearly return
    """
    returns = {}
    
    # Calculate monthly returns
    monthly_pct_change = closes.pct_change(fill_method=None).dropna() * 100
    
    for i, month in enumerate(MONTHS_ORDER[1:], 1):  # Skip 'Yearly'
        month_str = f"{year}-{i:02d}"
        month_returns = monthly_pct_change.loc[
            monthly_pct_change.index.strftime("%Y-%m") == month_str
        ]
        returns[month] = round(month_returns.values[0], 2) if not month_returns.empty else np.nan
    
    # Calculate yearly return
    dec_current = closes.loc[closes.index.strftime("%Y-%m") == f"{year}-12"]
    dec_previous = closes.loc[closes.index.strftime("%Y-%m") == f"{year-1}-12"]
    
    if not dec_current.empty and not dec_previous.empty:
        yearly_return = (dec_current.values[0] / dec_previous.values[0] - 1) * 100
        returns['Yearly'] = round(yearly_return, 2)
    else:
        returns['Yearly'] = np.nan
    
    return returns

# === WIDGET CREATION ===
index_options = ['All'] + list(INDICES.keys())

# Returns Tab Widgets
index_multiselect_returns = widgets.SelectMultiple(
    options=index_options,
    value=['Nifty 50'],
    description='Indices:',
    rows=4,
    disabled=False,
    style={'description_width': 'initial'}
)

years_list = [str(y) for y in range(DEFAULT_YEAR_RANGE[1], DEFAULT_YEAR_RANGE[0] - 1, -1)]
years_options = ['All'] + years_list
years_multiselect = widgets.SelectMultiple(
    options=years_options,
    value=years_list[:5],  # Default to last 5 years
    description='Years:',
    rows=4,
    disabled=False,
    style={'description_width': 'initial'}
)

returns_calculate_button = widgets.Button(
    description="Calculate Returns",
    style={'button_color': '#0072AA'},
    icon="bar-chart",
    tooltip="Calculate and display returns matrix"
)
returns_output = widgets.Output()

# Correlation Tab Widgets  
index_multiselect = widgets.SelectMultiple(
    options=index_options,
    rows=4,
    value=['All'],
    description='Indices:',
    disabled=False,
    style={'description_width': 'initial'}
)

interval_dropdown = widgets.Dropdown(
    options=[('Daily', 'daily'), ('Monthly', 'monthly')],
    value='daily',
    description='Interval:',
    disabled=False,
    style={'description_width': 'initial'}
)

price_dropdown = widgets.Dropdown(
    options=[('Open', 'open'), ('High', 'high'), ('Low', 'low'), ('Close', 'close')],
    value='close',
    description='Price Type:',
    disabled=False,
    style={'description_width': 'initial'}
)

start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=date.today() - timedelta(days=DEFAULT_LOOKBACK_DAYS),
    style={'description_width': 'initial'}
)

end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=date.today() - timedelta(days=1),
    style={'description_width': 'initial'}
)

calculate_button = widgets.Button(
    description="Calculate Correlation",
    style={'button_color': '#0072AA'},
    icon="calculator",
    tooltip="Calculate correlation matrix"
)
output = widgets.Output()

# === LAYOUT CREATION ===
returns_tab = widgets.VBox([
    widgets.HTML("<h3>Returns Analysis</h3>"),
    widgets.HBox([
        widgets.VBox([index_multiselect_returns], layout=widgets.Layout(width='33%')),
        widgets.VBox([years_multiselect], layout=widgets.Layout(width='33%')),
        widgets.VBox([returns_calculate_button], layout=widgets.Layout(width='33%', align_items='center'))
    ]),
    returns_output
])

correlation_tab = widgets.VBox([
    widgets.HTML("<h3>Correlation Matrix</h3>"),
    widgets.HBox([index_multiselect, interval_dropdown, price_dropdown, start_date_picker, end_date_picker, calculate_button]),
    output
])

tabs = widgets.Tab(children=[returns_tab, correlation_tab])
tabs.set_title(0, 'Returns Analysis')
tabs.set_title(1, 'Correlation Matrix')

display(tabs)

# === DATABASE CONNECTION ===
try:
    engine = create_engine('sqlite:///nse_data.db', echo=False)
    # Test connection
    test_query = "SELECT COUNT(*) FROM nse_ohlc_prices LIMIT 1"
    pd.read_sql(test_query, engine)
    # Logger message will not appear in notebook output
    logger.info("Database connection established successfully")
except Exception as e:
    logger.error(f"Database connection failed: {e}")
    print("⚠️  Database connection failed. Please run the data fetching section first to create the database.")

# === EVENT HANDLERS ===
def on_returns_calculate_clicked(button):
    """
    Handle returns calculation button click event.
    
    Args:
        button: Button widget that triggered the event
    """
    with returns_output:
        clear_output(wait=True)
        
        try:
            # Input validation and processing
            selected_indices = list(index_multiselect_returns.value)
            if 'All' in selected_indices:
                selected_indices = list(INDICES.keys())
            
            if not selected_indices:
                print("❌ Please select at least one index.")
                return

            years_selected = list(years_multiselect.value)
            if 'All' in years_selected:
                years_selected = years_list
            
            if not years_selected:
                print("❌ Please select at least one year.")
                return

            # Process years and build query parameters
            years_selected_int = sorted([int(y) for y in years_selected], reverse=True)
            min_year, max_year = min(years_selected_int), max(years_selected_int)
            query_start = f"{min_year-1}-12-01"
            query_end = f"{max_year}-12-31"
            
            selected_tickers = [INDICES[name] for name in selected_indices]
            
            
            # Optimized single query for all data
            query = f"""
                SELECT ticker, name, date, close 
                FROM nse_ohlc_prices 
                WHERE ticker IN ({build_query_parameters(selected_tickers)})
                AND date >= '{query_start}' AND date <= '{query_end}'
                ORDER BY ticker, date
            """
            
            df = pd.read_sql(query, engine, parse_dates=['date'])
            
            if df.empty:
                print("❌ No data found for selected indices and years.")
                return
            
            # Data quality validation
            data_issues = []
            for ticker in selected_tickers:
                ticker_data = df[df['ticker'] == ticker]
                # Find the index name for this ticker
                index_name = next((name for name, tick in INDICES.items() if tick == ticker), ticker)
                is_valid, issues = validate_data_quality(ticker_data, ticker, index_name)
                if not is_valid:
                    data_issues.extend(issues)
            
            df = df.set_index('date').sort_index()
            
            if len(selected_indices) == 1:
                # Single index analysis with monthly breakdown
                index_name = selected_indices[0]
                ticker = INDICES[index_name]
                index_data = df[df['ticker'] == ticker]['close']
                
                if index_data.empty:
                    print(f"❌ No data available for {index_name}")
                    return
                
                monthly_close = index_data.resample('ME').last()
                
                # Create returns grid
                grid = pd.DataFrame(index=[str(y) for y in years_selected_int], columns=MONTHS_ORDER)
                
                for year in years_selected_int:
                    year_closes = monthly_close.loc[
                        (monthly_close.index >= pd.Timestamp(f"{year-1}-12-01")) &
                        (monthly_close.index <= pd.Timestamp(f"{year}-12-31"))
                    ]
                    
                    if year_closes.empty:
                        continue
                    
                    # Handle current month exclusion if incomplete
                    if (year_closes.index.max().year == date.today().year and 
                        year_closes.index.max().month == date.today().month and
                        date.today().day < pd.Period(year_closes.index.max(), freq='M').days_in_month):
                        year_closes = year_closes.iloc[:-1]
                    
                    if not year_closes.empty:
                        returns_dict = calculate_monthly_returns(year_closes, year)
                        for month, return_val in returns_dict.items():
                            grid.loc[str(year), month] = return_val
                
                # Clean and display grid
                grid = grid.dropna(how='all').dropna(axis=1, how='all')
                grid_numeric = grid.apply(pd.to_numeric, errors='coerce')
                
                display(
                    grid_numeric.style
                    .format("{:.2f}%", na_rep="—")
                    .background_gradient(cmap="RdYlGn", axis=None, vmin=-10, vmax=10)
                    .set_caption(f"{index_name} - Monthly Returns Analysis")
                    .set_table_styles([
                        {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]}
                    ])
                )
                
                # Display data quality issues in collapsible section
                if data_issues:
                    create_collapsible_section("⚠️ Data Quality Issues", data_issues, is_open=False)
                
            else:
                # Multiple indices - yearly comparison
                yearly_grid = pd.DataFrame(index=[str(y) for y in years_selected_int], 
                                         columns=selected_indices)
                
                for index_name in selected_indices:
                    ticker = INDICES[index_name]
                    index_data = df[df['ticker'] == ticker]['close']
                    
                    if index_data.empty:
                        continue
                    
                    monthly_close = index_data.resample('ME').last()
                    
                    for year in years_selected_int:
                        year_closes = monthly_close.loc[
                            (monthly_close.index >= pd.Timestamp(f"{year-1}-12-01")) &
                            (monthly_close.index <= pd.Timestamp(f"{year}-12-31"))
                        ]
                        
                        if year_closes.empty:
                            yearly_grid.loc[str(year), index_name] = np.nan
                            continue
                        
                        returns_dict = calculate_monthly_returns(year_closes, year)
                        yearly_grid.loc[str(year), index_name] = returns_dict.get('Yearly', np.nan)
                
                # Clean and display grid
                yearly_grid = yearly_grid.dropna(how='all')
                yearly_grid_numeric = yearly_grid.apply(pd.to_numeric, errors='coerce')
                
            
                display(
                    yearly_grid_numeric.style
                    .format("{:.2f}%", na_rep="—")
                    .background_gradient(cmap="RdYlGn", axis=None, vmin=-20, vmax=30)
                    .set_caption("Yearly Returns Comparison Across Indices")
                    .set_table_styles([
                        {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]}
                    ])
                )
                
                # Display data quality issues in collapsible section
                if data_issues:
                    create_collapsible_section("⚠️ Data Quality Issues", data_issues, is_open=False)
                
        except Exception as e:
            logger.error(f"Error in returns calculation: {e}")
            print(f"❌ Error occurred during calculation: {str(e)}")
            print("Please check your data and try again.")

def on_correlation_calculate_clicked(button):
    """
    Handle correlation calculation button click event.
    
    Args:
        button: Button widget that triggered the event
    """
    with output:
        clear_output(wait=True)
        
        try:
            # Input validation
            selected_names = list(index_multiselect.value)
            if 'All' in selected_names:
                selected_names = list(INDICES.keys())
            
            if not selected_names:
                print("❌ Please select at least one index.")
                return
            
            selected_tickers = [INDICES[name] for name in selected_names]
            interval = interval_dropdown.value
            price_col = price_dropdown.value
            start = pd.Timestamp(start_date_picker.value)
            end = pd.Timestamp(end_date_picker.value)
            
            if start >= end:
                print("❌ Start date must be before end date.")
                return
            
            
            # Check data availability and adjust dates if necessary
            range_query = f"""
                SELECT ticker, MIN(date) as min_date, MAX(date) as max_date 
                FROM nse_ohlc_prices 
                WHERE ticker IN ({build_query_parameters(selected_tickers)})
                GROUP BY ticker
            """
            
            ranges = pd.read_sql(range_query, engine, parse_dates=['min_date', 'max_date'])
            
            if ranges.empty:
                print("❌ No data available for selected indices.")
                return
            
            # Find overlapping date range
            max_start = ranges['min_date'].max()
            min_end = ranges['max_date'].min()
            
            adjusted = False
            original_start, original_end = start, end
            
            if start < max_start:
                start = max_start
                start_date_picker.value = start.to_pydatetime().date()
                adjusted = True
                
            if end > min_end:
                end = min_end
                end_date_picker.value = end.to_pydatetime().date()
                adjusted = True
            
            if adjusted:
                print(f"⚠️ Date range adjusted to available data: {start.date()} to {end.date()}")
            
            # Fetch OHLC data
            data_query = f"""
                SELECT ticker, name, date, {price_col} 
                FROM nse_ohlc_prices 
                WHERE ticker IN ({build_query_parameters(selected_tickers)})
                AND date >= '{start.date()}' AND date <= '{end.date()}'
                ORDER BY date, ticker
            """
            
            df = pd.read_sql(data_query, engine, parse_dates=['date'])
            
            if df.empty:
                print("❌ No data found for the selected parameters.")
                return
            
            
            # Create price matrix
            prices = df.pivot(index='date', columns='name', values=price_col)
            
            # Collect data quality information
            quality_issues = []
            
            # Handle missing data
            missing_pct = (prices.isnull().sum() / len(prices) * 100).round(2)
            if missing_pct.any():
                for idx, pct in missing_pct[missing_pct > 0].items():
                    quality_issues.append(f"{idx}: {pct}% missing data")
            
            # Forward fill missing values
            prices = prices.ffill()
            
            # Remove columns with too much missing data (>20%)
            valid_columns = missing_pct[missing_pct <= 20].index
            if len(valid_columns) < len(prices.columns):
                removed_cols = set(prices.columns) - set(valid_columns)
                for col in removed_cols:
                    quality_issues.append(f"{col}: Removed due to >20% missing data ({missing_pct[col]:.1f}%)")
                prices = prices[valid_columns]
            
            if prices.empty:
                print("❌ No valid data remaining after quality checks.")
                return
            
            # Resample for monthly analysis
            if interval == 'monthly':
                prices = prices.resample('ME').last()
                # Remove current month if incomplete
                if end.day < pd.Period(end, freq='M').days_in_month:
                    prices = prices.iloc[:-1]
            
            # Calculate returns
            returns = prices.pct_change().dropna(how='all')
            returns.columns.name = None
            
            if returns.empty:
                print("❌ Insufficient data to calculate returns.")
                return
            
            # Remove extreme outliers (>3 standard deviations)
            outlier_info = []
            for col in returns.columns:
                mean_return = returns[col].mean()
                std_return = returns[col].std()
                outlier_threshold = 3 * std_return
                outliers = abs(returns[col] - mean_return) > outlier_threshold
                if outliers.any():
                    outlier_count = outliers.sum()
                    max_outlier = abs(returns[col] - mean_return).max()
                    outlier_info.append(f"{col}: {outlier_count} outliers removed (max deviation: {max_outlier:.3f})")
                    returns.loc[outliers, col] = np.nan
            
            quality_issues.extend(outlier_info)
            
            # Calculate correlation matrix
            corr = returns.corr()
            
           
            display(
                corr.style
                .background_gradient(cmap="RdYlGn", axis=None, vmin=-1, vmax=1)
                .format("{:.3f}")
                .set_caption(f"{interval.title()} Price Correlation Matrix ({price_col.title()} Prices)")
                .set_table_styles([
                    {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
                    {'selector': 'th', 'props': [('text-align', 'center')]},
                    {'selector': 'td', 'props': [('text-align', 'center')]}
                ])
            )
            
            # Display data quality issues in collapsible section
            if quality_issues:
                create_collapsible_section("📊 Data Quality Report", quality_issues, is_open=False)
            
            return corr
            
        except Exception as e:
            logger.error(f"Error in correlation calculation: {e}")
            print(f"❌ Error occurred during calculation: {str(e)}")
            print("Please check your inputs and try again.")

# === BIND EVENT HANDLERS ===
returns_calculate_button.on_click(on_returns_calculate_clicked)
calculate_button.on_click(on_correlation_calculate_clicked)

print("🚀 NSE Analytics Dashboard loaded successfully!")
print("📋 Features:")
print("  • Returns Analysis: Monthly/yearly returns matrices with data quality validation")
print("  • Correlation Analysis: Configurable correlation matrices with outlier detection")
print("  • Data Quality Checks: Automatic validation and cleaning of input data")
print("  • Interactive Interface: User-friendly widgets for parameter selection")

🚀 NSE Analytics Dashboard loaded successfully!
📋 Features:
  • Returns Analysis: Monthly/yearly returns matrices with data quality validation
  • Correlation Analysis: Configurable correlation matrices with outlier detection
  • Data Quality Checks: Automatic validation and cleaning of input data
  • Interactive Interface: User-friendly widgets for parameter selection
